<a href="https://colab.research.google.com/github/InannaxX07/Mutual-Fund-tracker/blob/main/Mutual_fund_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
from google.colab import files


print("Upload the September and November files.")
uploaded = files.upload()


file_september = "ZN250 - Monthly Portfolio September 2024.xlsx"
file_november = "ZN250 - Monthly Portfolio November 2024.xlsx"


def clean_data(df):
    columns_mapping = {
        "Unnamed: 2": "Instrument",
        "Unnamed: 3": "ISIN",
        "Unnamed: 4": "Sector",
        "Unnamed: 5": "Quantity",
        "Unnamed: 6": "Market Value (Rs. Lakhs)",
        "Unnamed: 7": "% to NAV"
    }
    df = df.rename(columns=columns_mapping)
    df = df[["Instrument", "ISIN", "Sector", "Quantity", "Market Value (Rs. Lakhs)", "% to NAV"]]
    df = df[df["Instrument"].notna() & ~df["Instrument"].str.contains("EQUITY|a\)", na=False)]
    return df.reset_index(drop=True)

# load and clean data
september_data = pd.read_excel(file_september, sheet_name="ZN250", skiprows=4)
november_data = pd.read_excel(file_november, sheet_name="ZN250", skiprows=4)

september_cleaned = clean_data(september_data)
november_cleaned = clean_data(november_data)

# add date columns for identification
september_cleaned["Date"] = "September 2024"
november_cleaned["Date"] = "November 2024"

combined = pd.concat([september_cleaned, november_cleaned], ignore_index=True)


pivot_data = combined.pivot_table(
    index=["Instrument", "ISIN", "Sector"],
    columns="Date",
    values=["Quantity", "Market Value (Rs. Lakhs)", "% to NAV"],
    aggfunc="first"
).reset_index()

# calculate changes
pivot_data["Change in Quantity"] = (
    pivot_data[("Quantity", "November 2024")] - pivot_data[("Quantity", "September 2024")]
)
pivot_data["Change in Market Value"] = (
    pivot_data[("Market Value (Rs. Lakhs)", "November 2024")] - pivot_data[("Market Value (Rs. Lakhs)", "September 2024")]
)
pivot_data["Change in % to NAV"] = (
    pivot_data[("% to NAV", "November 2024")] - pivot_data[("% to NAV", "September 2024")]
)

changes = pivot_data[
    (pivot_data["Change in Quantity"].notna() & (pivot_data["Change in Quantity"] != 0)) |
    (pivot_data["Change in Market Value"].notna() & (pivot_data["Change in Market Value"] != 0)) |
    (pivot_data["Change in % to NAV"].notna() & (pivot_data["Change in % to NAV"] != 0))
]


print("Changes in mutual fund allocations:")
print(changes)


Upload the September and November files.


Saving ZN250 - Monthly Portfolio November 2024.xlsx to ZN250 - Monthly Portfolio November 2024 (9).xlsx
Saving ZN250 - Monthly Portfolio September 2024.xlsx to ZN250 - Monthly Portfolio September 2024 (9).xlsx
Changes in mutual fund allocations:
                                             Instrument          ISIN  \
Date                                                                    
0                                      3M India Limited  INE470A01017   
1                                     ABB India Limited  INE117A01022   
2                                           ACC Limited  INE012A01025   
3                               AIA Engineering Limited  INE212H01026   
4                              APL Apollo Tubes Limited  INE702C01027   
..                                                  ...           ...   
243                                       Wipro Limited  INE075A01022   
244                                    Yes Bank Limited  INE528G01035   
245   ZF Commercial Vehi